# Assertion Checking for Transaction ID order

This notebook will check an ORESTAR-exported Excel transaction file to make sure that the transaction IDs are assigned in order of some date stored with the transaction.  So far, no such date has been found, but this is a WIP.

In [18]:
import pandas as pd
import numpy as np

Set `DATA_FILE` (below) to the path of the ORESTAR-provided Excel file you want to check.

Set `DATE_TO_TRY` to the (unicode) string name of the date column to check.

In [106]:
DATA_FILE = '../data/negatives.xlsx'
DATE_TO_TRY = u'Filed Date'
KEEP_ID = u'Original Id'

# Read the data file, convert it, and cherry pick the 
# potentially relevant columns.
df = pd.read_excel(
    DATA_FILE,
#     verbose=True,
).loc[:,[u'Tran Id',
         u'Original Id',
         u'Tran Date',
         u'Attest Date',
         u'Review Date',
         u'Due Date',
         u'Occptn Ltr Date',
         u'Filed Date',
         u'Exp Date']]

# brevity is boss
df = df.loc[:,[KEEP_ID, DATE_TO_TRY]]
df.rename(columns={KEEP_ID: 'id', DATE_TO_TRY: 'date'},
          inplace=True)

df.date = pd.to_datetime(df.date)
df = df.sort('date')

if sum(df.date.isnull()):
    raise Exception("Some of the {} values are null.".format(
            repr(DATE_TO_TRY)))

if sum(df.id.isnull()):
    raise Exception("Some of the {} values are null.".format(
            repr(KEEP_ID)))


In [107]:
max_ids = df[df.id == df.groupby('date')['id'].transform(max)]
max_ids.columns = ['max_id', 'date']

min_ids = df[df.id == df.groupby('date')['id'].transform(min)]
min_ids.columns = ['min_id', 'date']

min_max = min_ids.merge(max_ids, on='date', how='outer').sort('date')

In [108]:
if any((
    min_max.date.isnull().any(),
    min_max.max_id.isnull().any(),
    min_max.min_id.isnull().any(),
)):
    print sum(list(min_max.date.isnull()))
    print sum(list(min_max.max_id.isnull()))
    print sum(list(min_max.min_id.isnull()))
    raise Exception("I see null values where I shouldn't.")

## Ignore the warning that appears in red below the following cell

In [109]:
min_max['next_min'] = min_max.min_id.shift(-1)
min_max = min_max[:-1]
min_max['valid'] = min_max.max_id < min_max.next_min

In [110]:
mm_truths = min_max.valid.tolist()
if all(mm_truths):
    print "{} seems to follow the order of {}.".format(
    repr(DATE_TO_TRY), repr(KEEP_ID))
else:
    print ("In {}/{} cases, {} does NOT follow the order of\n" +
    "{} These are indicated with valid=False.\n\n").format(
        len(mm_truths) - sum(mm_truths),
        len(mm_truths),
        repr(DATE_TO_TRY),
        repr(KEEP_ID),
    )

    print min_max

In 28/331 cases, u'Filed Date' does NOT follow the order of
u'Original Id' These are indicated with valid=False.


      min_id       date   max_id  next_min  valid
0        198 2007-01-11      198      2280   True
1       2280 2007-01-15     2280      2770   True
2       2770 2007-01-22     2770      4923   True
3       4923 2007-01-24     4923      5469   True
4       5469 2007-01-25     5609      7209   True
5       7209 2007-01-27     7582      7949   True
6       7949 2007-01-28     7949      6202  False
7       6202 2007-01-29     8385      6351  False
8       6351 2007-01-30    12343      6769  False
9       6769 2007-01-31    15081     15366   True
10     15366 2007-02-01    15422     18314   True
11     18314 2007-02-02    18320      9262  False
12      9262 2007-02-03     9262       740  False
13       740 2007-02-05    19392      1909  False
14      1909 2007-02-07    22355      6079  False
15      6079 2007-02-12     6079     12733   True
16     12733 2007-02-14    12733   